In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install -q supervision
!pip install -q git+https://github.com/THU-MIG/yolov10.git
!pip install deep-sort-realtime
!pip install facenet-pytorch torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.2 MB/s eta 0

In [1]:
!mkdir -p weights
!wget -P weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10n.pt
!wget -P weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10s.pt
!wget -P weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10m.pt
!wget -P weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10b.pt
!wget -P weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10x.pt
!wget -P weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10l.pt
!ls -lh weights

total 408M
-rw-r--r-- 1 root root  80M May 23  2024 yolov10b.pt
-rw-r--r-- 1 root root 100M May 23  2024 yolov10l.pt
-rw-r--r-- 1 root root  64M May 23  2024 yolov10m.pt
-rw-r--r-- 1 root root  11M May 23  2024 yolov10n.pt
-rw-r--r-- 1 root root  32M May 23  2024 yolov10s.pt
-rw-r--r-- 1 root root 123M May 23  2024 yolov10x.pt


In [2]:
import cv2
import torch
import datetime
import numpy as np
from absl import app, flags
from absl.flags import FLAGS
from ultralytics import YOLOv10
from deep_sort_realtime.deepsort_tracker import DeepSort
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.models import Model
import joblib
from tensorflow.keras.preprocessing import image as keras_image
from scipy.spatial.distance import cosine

from facenet_pytorch import MTCNN, InceptionResnetV1
import torch



In [3]:
def extract_roi_from_frame(frame, x1, y1, x2, y2):
    """
    Extract the Region of Interest (ROI) from a video frame using bounding box coordinates.

    Parameters:
    - frame: The video frame.
    - x1, y1: Top-left corner of the bounding box.
    - x2, y2: Bottom-right corner of the bounding box.

    Returns:
    - roi: The cropped region from the frame.
    """
    # Validate bounding box coordinates
    if x1 < 0 or y1 < 0 or x2 > frame.shape[1] or y2 > frame.shape[0]:
        raise ValueError("Bounding box coordinates are out of frame bounds")

    roi = frame[y1:y2, x1:x2]

    if roi.size == 0:
        raise ValueError("Extracted ROI is empty")

    return roi


In [4]:
def get_feature_vector(roi):
    """
    Get the feature vector for a given ROI using a pre-trained VGG16 model.

    Parameters:
    - roi: The cropped region from the image.

    Returns:
    - feature_vector: The feature vector extracted from the ROI.
    """
    # Check if ROI is valid
    if roi is None or roi.size == 0:
        raise ValueError("Invalid ROI for feature extraction")

    # Resize ROI to the required input size for VGG16
    roi_resized = cv2.resize(roi, (224, 224))
    roi_array = keras_image.img_to_array(roi_resized)
    roi_array = np.expand_dims(roi_array, axis=0)
    roi_array = preprocess_input(roi_array)

    # Extract feature vector
    feature_vector = feature_model.predict(roi_array)
    return feature_vector.flatten()


In [5]:
# Function to store detections for a specific track_id
def store_detections(track_id, detection_data):
    if track_id not in tracked_objects:
        tracked_objects[track_id] = []
    tracked_objects[track_id].append(detection_data)


In [17]:
# Function to check if new detections match previous detections with a threshold
def match_with_previous_detections(current_detection_data):

  threshold=0.9
  # If track_id is not in unique, loop on all track IDs and their detections
  for track_id, detections in tracked_objects.items():
      for previous_detection_data in detections:
          if is_match(current_detection_data, previous_detection_data, threshold):
              return track_id

  return None


In [18]:
def is_match(current_detection_data, previous_detection_data, threshold):
    # Compare feature vectors
    feature_similarity = compare_feature_vectors(
        current_detection_data.get('feature_vector', None),
        previous_detection_data.get('feature_vector', None)
    )

    # Return True if the feature vector similarity is above the threshold
    return feature_similarity >= threshold


In [19]:
# Function to compare feature vectors using cosine similarity
def compare_feature_vectors(fv1, fv2):
    if fv1 is None or fv2 is None:
        return 0  # No similarity if either feature vector is missing

    return 1 - cosine(fv1, fv2)


In [28]:
import cv2
import numpy as np
import datetime
import torch
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image as keras_image
#from sklearn.externals import joblib
from deep_sort_realtime.deepsort_tracker import DeepSort

# Define constants
model_file = "/content/weights/yolov10x.pt"
video_file = "/content/videoplayback (online-video-cutter.com) (1).mp4"
output_file = "/content/output.mp4"
conf = 0.8
class_id = 0

# Dictionary to store tracking data for each track_id
tracked_objects = {}

# Set for unique track IDs
unique_track_ids = set()

# Load the SVM model
#svm_classifier = joblib.load('/content/model.pkl')
tracker = DeepSort(max_age=20, n_init=3)

# Initialize the video capture and writer
video_cap = cv2.VideoCapture(video_file)
frame_count = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

# Load YOLO model
model = YOLOv10(model_file)

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
feature_model = base_model

# Create colors for each class
np.random.seed(42)
colors = np.random.randint(0, 255, size=(80, 3))

# Process video frames
frame_idx = 0

while True:
    start_time = datetime.datetime.now()

    ret, frame = video_cap.read()
    if not ret:
        print("End of video file...")
        break

    # Perform object detection with YOLO
    results = model(frame, verbose=False)[0]
    detections = []

    for det in results.boxes:
        confidence = det.conf
        label = det.cls
        bbox = det.xyxy[0]
        x1, y1, x2, y2 = map(int, bbox)
        class_id = int(label)

        # Process only "person" class (class_id == 0) with high confidence
        if class_id == 0 and confidence >= conf:
            detections.append([[x1, y1, x2 - x1, y2 - y1], confidence, class_id])

    # Update tracker with person detections
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        width, height = x2 - x1, y2 - y1

        # Extract ROI and compute feature vector
        try:
            roi = extract_roi_from_frame(frame, x1, y1, x2, y2)
            feature_vector = get_feature_vector(roi)
        except ValueError as e:
            print(f"Error: {e}")
            continue

        current_detection_data = {
            'bounding_box': (x1, y1, x2, y2),
            'feature_vector': feature_vector
        }

        # Check for a match with previous detections
        matched_track_id = match_with_previous_detections(current_detection_data)
        if matched_track_id:
            track_id = matched_track_id
        else:
            unique_track_ids.add(track_id)

        # Store the detection
        store_detections(track_id, current_detection_data)

        # Draw bounding box and track ID on the frame
        color = colors[class_id]
        cv2.rectangle(frame, (x1, y1), (x2, y2), color.tolist(), 2)
        label = f"ID: {track_id}"
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color.tolist(), 2)

    # Write frame to output
    writer.write(frame)

    # Log processing time
    end_time = datetime.datetime.now()
    processing_time = (end_time - start_time).total_seconds()
    print(f"Processed frame {frame_idx + 1}/{frame_count} in {processing_time:.2f} seconds")
    frame_idx += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
video_cap.release()
writer.release()
cv2.destroyAllWindows()


Processed frame 1/292 in 4.32 seconds
Processed frame 2/292 in 2.15 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Processed frame 3/292 in 3.54 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 506ms/step
Processed frame 4/292 in 3.29 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step
Processed frame 5/292 in 2.54 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 511ms/step
Processed frame 6/292 in 2.72 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 519ms/step
Processed frame 7/292 in 2.66 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 949ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 647ms/step
Processed frame 8/292 in 4.62 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step
Processed frame 9/292 in 3.33 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 524ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 509ms/step
Processed frame 10/292 in 3.30 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 535ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 651ms/step
Processed frame 11/292 in 3.40 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 580ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 530ms/ste